In [ ]:
import sys
import os
# Append the library path to PYTHONPATH, so library can be imported.
sys.path.append(os.path.dirname(os.getcwd()))
import shutil

import numpy as np
import pandas as pd

from library import bs
from library import common as cm
from library import regression_aux as raux

In [ ]:
%run setup.py
%load_ext autoreload
%autoreload 2

In [ ]:
print('Loading data for:', FREQ)
print('Permutation flag:', PERMUTE)
print('VIX flag:', VIX)

In [ ]:
# load data must be after setup, because some of parameters are going to be overwritten.
%run Load_Clean_aux.py normal

In [ ]:
regr_dir = res_dir + 'Regression/'
print(f'Save results at:\n{regr_dir}')

In [ ]:
if os.path.exists(regr_dir):
    shutil.rmtree(regr_dir)
os.makedirs(regr_dir)
shutil.copy(f'setup.py', regr_dir) 

In [ ]:
def save_tables(res, dirs):
    if res is not None:
        res['df_coef'].to_csv(f'{dirs}coef.csv')
        res['df_fit_std'].to_csv(f'{dirs}std.csv')

### NO permute

#### No hedge

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}No_Hedge/'
    os.makedirs(f'{sub_regr_dir}pnl/', exist_ok=True)
    
    for i in range(NUM_TEST):
        df_test = mc_sets[i]
        df_tmp = df_train.append(df_test)
        zero = np.array([0.]*len(df_tmp))
        zero = pd.Series(zero, index=df_tmp.index)
        pnl_path = f'{sub_regr_dir}pnl/pnl{i}.csv'
        cm.store_pnl(
            df_tmp, zero,
            pnl_path=pnl_path)

#### BS

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}BS_Benchmark/'
    os.makedirs(f'{sub_regr_dir}pnl/', exist_ok=True)
    for i in range(NUM_TEST):
        df_test = mc_sets[i]
        df_tmp = df_train.append(df_test)
        pnl_path = f'{sub_regr_dir}pnl/pnl{i}.csv'
        cm.store_pnl(
            df_tmp, df_tmp['delta_bs'],
            pnl_path=pnl_path)

#### Alexander 2007 minimum-variance strategy
$$ \delta_{HS} + \mathcal{V}_{HS} \frac{\rho \sigma}{S} $$
Here, $\rho$ is the correlation between the two Brownian motions in Heston model, $\sigma$ is the volatility of variance process. The $\mathcal{V}_{HS}$ is not the Vega in the normal sense. It's the sensitivity of price to variance, not to volatility.

In [ ]:
if UNDERLYING_MODEL == 'Heston':
    if not PERMUTE:
        sub_regr_dir = f'{regr_dir}Alexander2007/'
        os.makedirs(f'{sub_regr_dir}pnl/', exist_ok=True)
        for i in range(NUM_TEST):
            df_test = mc_sets[i]
            df_tmp = df_train.append(df_test)
            pnl_path = f'{sub_regr_dir}pnl/pnl{i}.csv'
            strat = df_tmp['delta_hs'] + \
                    df_tmp['vega_hs_n'] * UNDERLYINGPARAS['rho'] * UNDERLYINGPARAS['sigma'] / df_tmp['S0_n'] # * 1 / (2*np.sqrt(df_tmp['Var0']))
            cm.store_pnl(
                df_tmp, strat,
                pnl_path=pnl_path)

#### Add ATM option as the second instrument
We use an ATM one-month option as the second instrument. Denote the number of ATM option to hold by $\eta$, and that of underlying by $\Delta$.
We propose a strategy such that 
$$ \eta  \mathcal{V}_{ATM} - \mathcal{V}_{TH} = 0, \\
    \eta  \delta_{ATM} - \delta_{TH} + \Delta = 0. $$
Here $\mathcal{V}_{ATM}$ and $\mathcal{V}_{TH}$ are the Sensitivity of the ATM option and the option to be hedged (TH) w.r.t variance, not to volatility. They, as well as $\delta_{TH}$, are calulated by the finite difference of Heston pricing formula, rather than using the BS with implied vol.
Hence, 
$$ \eta = \frac{\mathcal{V}_{TH}}{\mathcal{V}_{ATM}} \\
\Delta = \delta_{TH} - \eta \delta_{ATM} $$ 

In [ ]:
if UNDERLYING_MODEL == 'Heston':
    if not PERMUTE:
        sub_regr_dir = f'{regr_dir}Two_Assets/'
        os.makedirs(f'{sub_regr_dir}pnl/', exist_ok=True)
        for i in range(NUM_TEST):
            df_test = mc_sets[i]
            df_tmp = df_train.append(df_test)
            pnl_path = f'{sub_regr_dir}pnl/pnl{i}.csv'
            eta = df_tmp['vega_hs_n'] / df_tmp['vega_hs_atm_n']
            delta = df_tmp['delta_hs'] - eta * df_tmp['delta_hs_atm']

            cm.store_pnl_two_assets(
                df_tmp, delta, eta,
                pnl_path=pnl_path)

#### Delta-only

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Delta_only/'
    for i in range(NUM_TEST):
        df_test = mc_sets[i]
        df_tmp = df_train.append(df_test)
        kwargs = {'vix': VIX, 
                  'features': ['delta_bs'], 
                  'max_period': 0, 
                  'sub_res': f'{sub_regr_dir}pnl/',
                  'pnl_path': f'{sub_regr_dir}pnl/pnl{i}.csv',
                  'df': df_tmp,
                  'delta_coeff_1': False,
                  'agg_side': False,
                  'leverage': True
                 }
        res = raux.run_store_lin(**kwargs)
    save_tables(res, dirs=sub_regr_dir)

#### Delta-Vega

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Delta_Vega/'
    for i in range(NUM_TEST):
        df_test = mc_sets[i]
        df_tmp = df_train.append(df_test)
        kwargs = {'vix': VIX, 
                  'features': ['delta_bs', 'vega_n'], 
                  'max_period': 0, 
                  'sub_res': f'{sub_regr_dir}pnl/',
                  'pnl_path': f'{sub_regr_dir}pnl/pnl{i}.csv',
                  'df': df_tmp,
                  'delta_coeff_1': False,
                  'agg_side': False
                 }
        res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

#### Delta-Gamma

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Delta_Gamma/'
    for i in range(NUM_TEST):
        df_test = mc_sets[i]
        df_tmp = df_train.append(df_test)
        kwargs = {'vix': VIX, 
                  'features': ['delta_bs', 'gamma_n'], 
                  'max_period': 0, 
                  'sub_res': f'{sub_regr_dir}pnl/',
                  'pnl_path': f'{sub_regr_dir}pnl/pnl{i}.csv',
                  'df': df_tmp,
                  'delta_coeff_1': False,
                  'agg_side': False
                 }
        res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

#### Delta-Vanna

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Delta_Vanna/'
    for i in range(NUM_TEST):
        df_test = mc_sets[i]
        df_tmp = df_train.append(df_test)
        kwargs = {'vix': VIX, 
                  'features': ['delta_bs', 'vanna_n'], 
                  'max_period': 0, 
                  'sub_res': f'{sub_regr_dir}pnl/',
                  'pnl_path': f'{sub_regr_dir}pnl/pnl{i}.csv',
                  'df': df_tmp,
                  'delta_coeff_1': False,
                  'agg_side': False
                 }
        res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

####  Delta-Gamma-Vanna

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Delta_Gamma_Vanna/'
    for i in range(NUM_TEST):
        df_test = mc_sets[i]
        df_tmp = df_train.append(df_test)
        kwargs = {'vix': VIX, 
                  'features': ['delta_bs', 'gamma_n', 'vanna_n'], 
                  'max_period': 0, 
                  'sub_res': f'{sub_regr_dir}pnl/',
                  'pnl_path': f'{sub_regr_dir}pnl/pnl{i}.csv',
                  'df': df_tmp,
                  'delta_coeff_1': False,
                  'agg_side': False
                 }
        res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

#### Delta-Vega-Gamma

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Delta_Vega_Gamma/'
    for i in range(NUM_TEST):
        df_test = mc_sets[i]
        df_tmp = df_train.append(df_test)
        kwargs = {'vix': VIX, 
                  'features': ['delta_bs', 'vega_n', 'gamma_n'], 
                  'max_period': 0, 
                  'sub_res': f'{sub_regr_dir}pnl/',
                  'pnl_path': f'{sub_regr_dir}pnl/pnl{i}.csv',
                  'df': df_tmp,
                  'delta_coeff_1': False,
                  'agg_side': False
                 }
        res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

#### Delta-Vega-Vanna

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Delta_Vega_Vanna/'
    for i in range(NUM_TEST):
        df_test = mc_sets[i]
        df_tmp = df_train.append(df_test)
        kwargs = {'vix': VIX, 
                  'features': ['delta_bs', 'vega_n', 'vanna_n'], 
                  'max_period': 0, 
                  'sub_res': f'{sub_regr_dir}pnl/',
                  'pnl_path': f'{sub_regr_dir}pnl/pnl{i}.csv',
                  'df': df_tmp,
                  'delta_coeff_1': False,
                  'agg_side': False
                 }
        res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

#### Delta-Vega-Gamma-Vanna

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Delta_Vega_Gamma_Vanna/'
    for i in range(NUM_TEST):
        df_test = mc_sets[i]
        df_tmp = df_train.append(df_test)
        kwargs = {'vix': VIX, 
                  'features': ['delta_bs', 'vega_n', 'gamma_n', 'vanna_n'], 
                  'max_period': 0, 
                  'sub_res': f'{sub_regr_dir}pnl/',
                  'pnl_path': f'{sub_regr_dir}pnl/pnl{i}.csv',
                  'df': df_tmp,
                  'delta_coeff_1': False,
                  'agg_side': False
                 }
        res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

#### Vanna-only

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Vanna_only/'
    for i in range(NUM_TEST):
        df_test = mc_sets[i]
        df_tmp = df_train.append(df_test)
        kwargs = {'vix': VIX, 
                  'features': ['vanna_n'], 
                  'max_period': 0, 
                  'sub_res': f'{sub_regr_dir}pnl/',
                  'pnl_path': f'{sub_regr_dir}pnl/pnl{i}.csv',
                  'df': df_tmp,
                  'delta_coeff_1': True,
                  'agg_side': False,
                  'leverage': False
                 }
        res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

#### Vega-only

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Vega_only/'
    for i in range(NUM_TEST):
        df_test = mc_sets[i]
        df_tmp = df_train.append(df_test)
        kwargs = {'vix': VIX, 
                  'features': ['vega_n'], 
                  'max_period': 0, 
                  'sub_res': f'{sub_regr_dir}pnl/',
                  'pnl_path': f'{sub_regr_dir}pnl/pnl{i}.csv',
                  'df': df_tmp,
                  'delta_coeff_1': True,
                  'agg_side': False,
                  'leverage': False
                 }
        res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

#### Gamma-only

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Gamma_only/'
    for i in range(NUM_TEST):
        df_test = mc_sets[i]
        df_tmp = df_train.append(df_test)
        kwargs = {'vix': VIX, 
                  'features': ['gamma_n'], 
                  'max_period': 0, 
                  'sub_res': f'{sub_regr_dir}pnl/',
                  'pnl_path': f'{sub_regr_dir}pnl/pnl{i}.csv',
                  'df': df_tmp,
                  'delta_coeff_1': True,
                  'agg_side': False,
                  'leverage': False
                 }
        res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

#### Bias only regression

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Bias/'
    
    for i in range(NUM_TEST):
        df_test = mc_sets[i]
        df_tmp = df_train.append(df_test)
        df_tmp['bias'] = 1.
        kwargs = {'vix': VIX, 
                  'features': ['bias'], 
                  'max_period': 0, 
                  'sub_res': f'{sub_regr_dir}pnl/',
                  'pnl_path': f'{sub_regr_dir}pnl/pnl{i}.csv',
                  'df': df_tmp,
                  'delta_coeff_1': True,
                  'agg_side': False,
                  'leverage': False
                 }
        res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

#### Hull White model

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Hull_White/'
    for i in range(NUM_TEST):
        df_test = mc_sets[i]
        df_tmp = df_train.append(df_test)
        kwargs = {'vix': VIX, 
                  'features': ['vega_s', 'delta_vega_s', 'delta2_vega_s'], 
                  'max_period': 0, 
                  'sub_res': f'{sub_regr_dir}pnl/',
                  'pnl_path': f'{sub_regr_dir}pnl/pnl{i}.csv',
                  'df': df_tmp,
                  'delta_coeff_1': True,
                  'agg_side': False
                 }
        res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

In [ ]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Hull_White_relaxed/'
    for i in range(NUM_TEST):
        df_test = mc_sets[i]
        df_tmp = df_train.append(df_test)
        kwargs = {'vix': VIX, 
                  'features': ['delta_bs', 'vega_s', 'delta_vega_s', 'delta2_vega_s'], 
                  'max_period': 0, 
                  'sub_res': f'{sub_regr_dir}pnl/',
                  'pnl_path': f'{sub_regr_dir}pnl/pnl{i}.csv',
                  'df': df_tmp,
                  'delta_coeff_1': False,
                  'agg_side': False
                 }
        res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

### Permute

#### Prepare all permuted data sets

In [ ]:
if PERMUTE:
    train_permutes, test_permutes = [], []
    for i in range(NUM_TEST):
        # the union of train and test
        df_permute = df_train.append(mc_sets[i], ignore_index=True, sort=False)
        df_permute['Is_In_Some_Test'] = False
        df_permute = cm.permute_core(df_permute, 0, random_seed=i)

        df_train_permuted = df_permute.loc[((df_permute['period0'] == 0) | (df_permute['period0'] == 1))]
        df_test_permuted = df_permute.loc[df_permute['period0'] == 2]

        train_permutes.append(df_train_permuted.copy())
        test_permutes.append(df_test_permuted.copy())

#### No hedge

In [ ]:
if PERMUTE:
    print('Permuted!')
    sub_res = f'{regr_dir}No_Hedge/pnl/'
    os.makedirs(sub_res, exist_ok=True)
    for i in range(NUM_TEST):
        df_tmp = train_permutes[i].append(test_permutes[i])
        zero = np.array([0.]*len(df_tmp))
        zero = pd.Series(zero, index=df_tmp.index)
        cm.store_pnl(
            df_tmp, zero,
            pnl_path=f'{sub_res}pnl{i}.csv')

#### BS
Permuting affects the Black-Scholes, because the out-of-sample set is different.

In [ ]:
if PERMUTE:
    print('Permuted!')
    sub_res = f'{regr_dir}BS_Benchmark/pnl/'
    os.makedirs(sub_res, exist_ok=True)
    for i in range(NUM_TEST):
        df_tmp = train_permutes[i].append(test_permutes[i])
        cm.store_pnl(
            df_tmp, df_tmp['delta_bs'],
            pnl_path=f'{sub_res}pnl{i}.csv')

#### Delta-Vega-Vanna

In [ ]:
if PERMUTE:
    print('Permuted!')

    for i in range(NUM_TEST):    
        df_tmp = train_permutes[i].append(test_permutes[i])
        kwargs = {'vix': VIX, 
              'features': ['delta_bs', 'vega_n', 'vanna_n'], 
              'max_period': 0, 
              'sub_res': f'{regr_dir}Delta_Vega_Vanna/pnl/',
              'pnl_path': f'{regr_dir}Delta_Vega_Vanna/pnl/pnl{i}.csv',
              'df': df_tmp,
              'delta_coeff_1': False,
              'agg_side': False
         }
        res = raux.run_store_lin(**kwargs)
        print(res['df_coef'])